# Convert raw folders from ISAS to NIfTI

This notebook attempts to convert raw folders (including .dcm files for CT, MR, registration and RTSS) into NIfTI files. 

In [1]:
import os
import glob

# array manipulation and plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# medical image manipulation 
import SimpleITK as sitk

from DicomRTTool.ReaderWriter import DicomReaderWriter

In [16]:
DICOM_path = os.path.join('/home/akamath/Desktop/ssn_raw/ISAS_GBM_007/') # folder where downloaded data was stored
print(DICOM_path)

Dicom_reader = DicomReaderWriter(description='Examples', arg_max=True)
Dicom_reader.walk_through_folders(DICOM_path) # need to define in order to use all_roi method

/home/akamath/Desktop/ssn_raw/ISAS_GBM_007/


Loading through DICOM files:   0%|          | 0/1 [00:00<?, ?it/s]

Loading from /home/akamath/Desktop/ssn_raw/ISAS_GBM_007/


Loading through DICOM files: 100%|██████████| 1/1 [00:00<00:00,  3.61it/s]

Compiling dictionaries together...
Index 0, description ARIA RadOnc Images - Changed patient position  at /home/akamath/Desktop/ssn_raw/ISAS_GBM_007/
Index 1, description None at /home/akamath/Desktop/ssn_raw/ISAS_GBM_007/
Index 2, description Tirm_cor_TI_2500 at /home/akamath/Desktop/ssn_raw/ISAS_GBM_007/
Index 3, description t1_se_tra  at /home/akamath/Desktop/ssn_raw/ISAS_GBM_007/
Index 4, description t2_spc_sag_p2_iso  at /home/akamath/Desktop/ssn_raw/ISAS_GBM_007/
5 unique series IDs were found. Default is index 0, to change use set_index(index)


In [29]:
all_rois = Dicom_reader.return_rois(print_rois=True)  # Return a list of all rois present, and print them
indexes = Dicom_reader.which_indexes_have_all_rois()  # Check to see which indexes have all of the rois we want, now we can see indexes

Contour_Names = ['tumor', 'high_dose'] 
# Associations work as {'variant_name': 'desired_name'}
associations = {'dose 1000[cgy]': 'high_dose', 'dose 1200[cgy]': 'high_dose', # new high dose ROI through association
               'exprtfrontrecgbm': 'tumor', 'rtfrontrecgbm': 'tumor', 'expltfrontgbm': 'tumor', # associating gbms to tumor 
                'ltfrontgbm': 'tumor','infrttempgbm': 'tumor', 'rttemprecgbm': 'tumor',
               'exprttemprecgbm': 'tumor','expltfrparrecgbm': 'tumor', 'ltfrparrecgbm': 'tumor'} 

Dicom_reader.set_contour_names_and_associations(Contour_Names=Contour_Names, associations=associations)

pt_indx = 4 #indexes[-1]
Dicom_reader.set_index(pt_indx)  # This index has all the structures, corresponds to pre-RT T1-w image for patient 011
Dicom_reader.get_images_and_mask()  # Load up the images and mask for the requested index

The following ROIs were found
r_brainstem
r_hippocampus_l
r_hippocampus_r
r_hippocampus_r2
r_hippocampus_l2
r_brainstem1
r_hippocampus_r1
r_hippocampus_l1
edema
enhancing_tumor
resection_cavity
gtvp
The following indexes have all ROIs present
Finished listing present indexes
Lacking ['tumor', 'high_dose'] in index 0, location /home/akamath/Desktop/ssn_raw/ISAS_GBM_007/. Found ['r_brainstem', 'r_hippocampus_l', 'r_hippocampus_r', 'r_hippocampus_r2', 'r_hippocampus_l2', 'r_brainstem1', 'r_hippocampus_r1', 'r_hippocampus_l1', 'edema', 'enhancing_tumor', 'resection_cavity', 'gtvp']
Lacking ['tumor', 'high_dose'] in index 1, location /home/akamath/Desktop/ssn_raw/ISAS_GBM_007/. Found []
Lacking ['tumor', 'high_dose'] in index 2, location /home/akamath/Desktop/ssn_raw/ISAS_GBM_007/. Found []
Lacking ['tumor', 'high_dose'] in index 3, location /home/akamath/Desktop/ssn_raw/ISAS_GBM_007/. Found []
Lacking ['tumor', 'high_dose'] in index 4, location /home/akamath/Desktop/ssn_raw/ISAS_GBM_007/. 

In [30]:
dicom_sitk_handle = Dicom_reader.dicom_handle # SimpleITK image handle
mask_sitk_handle = Dicom_reader.annotation_handle # SimpleITK mask handle

sitk.WriteImage(dicom_sitk_handle, os.path.join('/home/akamath/Desktop/ssn_raw/', 'Image.nii'))
sitk.WriteImage(mask_sitk_handle, os.path.join('/home/akamath/Desktop/ssn_raw/', 'Mask.nii'))

In [36]:
dcm_003_00 = sitk.ReadImage("/home/akamath/Desktop/ssn_raw/GBM_003/Image_01.nii")
print(dcm_003_00.GetOrigin())
print(dcm_003_00.GetSpacing())


(-71.4487533569336, -85.14421844482422, 110.96417236328125)
(0.6875, 0.6875, 4.799999713897705)
